In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df = pd.read_csv("Titanic-Dataset.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


# **INGENIERIA DE FEATURES**

## 2.1 Creación de Variables Derivadas

### Variable Title

In [ ]:
# --- 1. VARIABLE: Title (TITULO) ---

def title_feature(df):
    df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    df['Title'] = df['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    df['Title'] = df['Title'].replace('Mlle', 'Miss')
    df['Title'] = df['Title'].replace('Ms', 'Miss')
    df['Title'] = df['Title'].replace('Mme', 'Mrs')
    # ANÁLISIS DE LA RELACIÓN CON SUPERVIVENCIA
    print("RELACIÓN DE 'Title' CON LA SUPERVIVENCIA:")
    print(df.groupby('Title')['Survived'].mean().sort_values(ascending=False))
    # VERIFICACIÓN DE CALIDAD
    print("\nVERIFICACIÓN DE VALORES ÚNICOS Y CONTEO:")
    print(df['Title'].value_counts())

title_feature(df)


RELACIÓN DE 'Title' CON LA SUPERVIVENCIA:
Title
Mrs       0.793651
Miss      0.702703
Master    0.575000
Rare      0.347826
Mr        0.156673
Name: Survived, dtype: float64

VERIFICACIÓN DE VALORES ÚNICOS Y CONTEO:
Title
Mr        517
Miss      185
Mrs       126
Master     40
Rare       23
Name: count, dtype: int64


<>:4: SyntaxWarning: invalid escape sequence '\.'
<>:4: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipython-input-199068824.py:4: SyntaxWarning: invalid escape sequence '\.'
  df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)


### Variable Family Size


In [ ]:
# --- 2. VARIABLE: FamilySize (TAMAÑO DE LA FAMILIA) ---

def family_size_feature(df):
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    # ANÁLISIS DE LA RELACIÓN CON SUPERVIVENCIA
    print("RELACIÓN DE 'FamilySize' CON LA SUPERVIVENCIA:")
    print(df.groupby('FamilySize')['Survived'].mean())
    # VERIFICACIÓN DE CALIDAD
    print("\nVERIFICACIÓN DE VALORES ÚNICOS Y CONTEO:")
    print(df['FamilySize'].value_counts().sort_index())

family_size_feature(df)

RELACIÓN DE 'FamilySize' CON LA SUPERVIVENCIA:
FamilySize
1     0.303538
2     0.552795
3     0.578431
4     0.724138
5     0.200000
6     0.136364
7     0.333333
8     0.000000
11    0.000000
Name: Survived, dtype: float64

VERIFICACIÓN DE VALORES ÚNICOS Y CONTEO:
FamilySize
1     537
2     161
3     102
4      29
5      15
6      22
7      12
8       6
11      7
Name: count, dtype: int64


### Variable IsAlone

In [ ]:
# --- 3. VARIABLE: IsAlone (ESTA SOLO) ---

def is_alone_feature(df):
    df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
    # ANÁLISIS DE LA RELACIÓN CON SUPERVIVENCIA
    print("RELACIÓN DE 'IsAlone' CON LA SUPERVIVENCIA:")
    print(df.groupby('IsAlone')['Survived'].mean())
    # VERIFICACIÓN DE CALIDAD
    print("\nVERIFICACIÓN DE VALORES ÚNICOS Y CONTEO:")
    print(df['IsAlone'].value_counts())

is_alone_feature(df)

RELACIÓN DE 'IsAlone' CON LA SUPERVIVENCIA:
IsAlone
0    0.505650
1    0.303538
Name: Survived, dtype: float64

VERIFICACIÓN DE VALORES ÚNICOS Y CONTEO:
IsAlone
1    537
0    354
Name: count, dtype: int64


### Variable AgeGroup

In [ ]:
# --- 4. VARIABLE: AgeGroup (GRUPO DE EDAD) ---
# MANEJO DE VALORES NULOS PARA LA EDAD

def age_group_feature(df):
      df['Age'].fillna(df['Age'].median(), inplace=True)
      bins = [0, 12, 18, 60, np.inf]
      labels = ['Child', 'Adolescent/Teenager', 'Adult', 'Senior']
      df['AgeGroup'] = pd.cut(df['Age'], bins=bins, labels=labels, right=False)
      # ANÁLISIS DE LA RELACIÓN CON SUPERVIVENCIA
      print("RELACIÓN DE 'AgeGroup' CON LA SUPERVIVENCIA:")
      print(df.groupby('AgeGroup')['Survived'].mean().sort_values(ascending=False))
      # VERIFICACIÓN DE CALIDAD
      print("\nVERIFICACIÓN DE VALORES ÚNICOS Y CONTEO:")
      print(df['AgeGroup'].value_counts())

age_group_feature(df)


RELACIÓN DE 'AgeGroup' CON LA SUPERVIVENCIA:
AgeGroup
Child                  0.573529
Adolescent/Teenager    0.488889
Adult                  0.364362
Senior                 0.269231
Name: Survived, dtype: float64

VERIFICACIÓN DE VALORES ÚNICOS Y CONTEO:
AgeGroup
Adult                  752
Child                   68
Adolescent/Teenager     45
Senior                  26
Name: count, dtype: int64


/tmp/ipython-input-2513141969.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)
/tmp/ipython-input-2513141969.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby('AgeGroup')['Survived'].mean().sort_values(ascending=False))


### Variable FarePerPerson

In [ ]:
# --- 5. VARIABLE: FarePerPerson (TARIFA POR PERSONA) ---

def fare_per_person_feature(df):
    df['FarePerPerson'] = df['Fare'] / df['FamilySize']
    df['FarePerPerson'].fillna(df['FarePerPerson'].mean(), inplace=True) # Manejo de Nulos
    # ANÁLISIS DE LA RELACIÓN CON SUPERVIVENCIA (AGRUPANDO EN RANGOS)
    print("RELACIÓN DE 'FarePerPerson' CON LA SUPERVIVENCIA (POR RANGOS):")
    df['FarePerPerson_Group'] = pd.qcut(df['FarePerPerson'], 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
    print(df.groupby('FarePerPerson_Group')['Survived'].mean().sort_values(ascending=False))
    # VERIFICACIÓN DE CALIDAD
    print("\nVERIFICACIÓN DE LA DISTRIBUCIÓN:")
    print(df['FarePerPerson'].describe())

fare_per_person_feature(df)


RELACIÓN DE 'FarePerPerson' CON LA SUPERVIVENCIA (POR RANGOS):
FarePerPerson_Group
Q4    0.608108
Q3    0.408072
Q1    0.265487
Q2    0.254545
Name: Survived, dtype: float64

VERIFICACIÓN DE LA DISTRIBUCIÓN:
count    891.000000
mean      19.916375
std       35.841257
min        0.000000
25%        7.250000
50%        8.300000
75%       23.666667
max      512.329200
Name: FarePerPerson, dtype: float64


/tmp/ipython-input-897968984.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['FarePerPerson'].fillna(df['FarePerPerson'].mean(), inplace=True) # Manejo de Nulos
/tmp/ipython-input-897968984.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby('FarePerPerson_Group')['Survived'].mean().sort_values(ascending=False))


### Variable Cabin Deck

In [ ]:
# --- 6. VARIABLE: CabinDeck (CUBIERTA DE LA CABINA) ---

def cabin_deck_feature(df):
    df['CabinDeck'] = df['Cabin'].str.extract('([A-Z])', expand=False)
    df['CabinDeck'].fillna('Unknown', inplace=True)
    # ANÁLISIS DE LA RELACIÓN CON SUPERVIVENCIA
    print("RELACIÓN DE 'CabinDeck' CON LA SUPERVIVENCIA:")
    print(df.groupby('CabinDeck')['Survived'].mean().sort_values(ascending=False))
    # VERIFICACIÓN DE CALIDAD
    print("\nVERIFICACIÓN DE VALORES ÚNICOS Y CONTEO:")
    print(df['CabinDeck'].value_counts())

cabin_deck_feature(df)

RELACIÓN DE 'CabinDeck' CON LA SUPERVIVENCIA:
CabinDeck
D          0.757576
E          0.750000
B          0.744681
F          0.615385
C          0.593220
G          0.500000
A          0.466667
Unknown    0.299854
T          0.000000
Name: Survived, dtype: float64

VERIFICACIÓN DE VALORES ÚNICOS Y CONTEO:
CabinDeck
Unknown    687
C           59
B           47
D           33
E           32
A           15
F           13
G            4
T            1
Name: count, dtype: int64


/tmp/ipython-input-3112934591.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['CabinDeck'].fillna('Unknown', inplace=True)


### Variable CabinKnown

In [ ]:
# --- 7. VARIABLE: CabinKnown (CABINA CONOCIDA) ---

def cabin_known_feature(df):
  df['CabinKnown'] = df['Cabin'].isnull().astype(int)
  # ANÁLISIS DE LA RELACIÓN CON SUPERVIVENCIA
  print("RELACIÓN DE 'CabinKnown' CON LA SUPERVIVENCIA:")
  print(df.groupby('CabinKnown')['Survived'].mean())
  # VERIFICACIÓN DE CALIDAD
  print("\nVERIFICACIÓN DE VALORES ÚNICOS Y CONTEO:")
  print(df['CabinKnown'].value_counts())

cabin_known_feature(df)


RELACIÓN DE 'CabinKnown' CON LA SUPERVIVENCIA:
CabinKnown
0    0.666667
1    0.299854
Name: Survived, dtype: float64

VERIFICACIÓN DE VALORES ÚNICOS Y CONTEO:
CabinKnown
1    687
0    204
Name: count, dtype: int64


### Variable TicketFrequency

In [ ]:
# --- 8. VARIABLE: TicketFrequency (FREQUENCIA DE TICKEY ) ---

def ticket_frequency_feature(df):
  df['TicketFrequency'] = df.groupby('Ticket')['Ticket'].transform('count')
  # ANÁLISIS DE LA RELACIÓN CON SUPERVIVENCIA
  print("RELACIÓN DE 'TicketFrequency' CON LA SUPERVIVENCIA:")
  print(df.groupby('TicketFrequency')['Survived'].mean())
  # VERIFICACIÓN DE CALIDAD
  print("\nVERIFICACIÓN DE VALORES ÚNICOS Y CONTEO:")
  print(df['TicketFrequency'].value_counts().sort_index())

ticket_frequency_feature(df)

RELACIÓN DE 'TicketFrequency' CON LA SUPERVIVENCIA:
TicketFrequency
1    0.297989
2    0.574468
3    0.698413
4    0.500000
5    0.000000
6    0.000000
7    0.238095
Name: Survived, dtype: float64

VERIFICACIÓN DE VALORES ÚNICOS Y CONTEO:
TicketFrequency
1    547
2    188
3     63
4     44
5     10
6     18
7     21
Name: count, dtype: int64


### Variable NameLength

In [ ]:
# --- 9. VARIABLE: NameLength (LONGITUD DE NOMBRE ) ---

def name_length_feature(df):
  df['NameLength'] = df['Name'].str.len()
  # ANÁLISIS DE LA RELACIÓN CON SUPERVIVENCIA
  print("RELACIÓN DE 'NameLength' CON LA SUPERVIVENCIA:")
  print(df.groupby('NameLength')['Survived'].mean())
  # VERIFICACIÓN DE CALIDAD
  print("\nVERIFICACIÓN DE VALORES ÚNICOS Y CONTEO:")
  print(df['NameLength'].value_counts().sort_index())

name_length_feature(df)


RELACIÓN DE 'NameLength' CON LA SUPERVIVENCIA:
NameLength
12    0.500000
13    0.500000
14    0.333333
15    0.133333
16    0.230769
17    0.214286
18    0.200000
19    0.234375
20    0.282051
21    0.325000
22    0.315789
23    0.282051
24    0.372093
25    0.327273
26    0.224490
27    0.360000
28    0.372093
29    0.500000
30    0.432432
31    0.400000
32    0.565217
33    0.545455
34    0.428571
35    1.000000
36    0.333333
37    0.700000
38    0.444444
39    0.444444
40    0.428571
41    1.000000
42    0.200000
43    0.800000
44    1.000000
45    0.777778
46    0.571429
47    0.727273
48    1.000000
49    1.000000
50    1.000000
51    1.000000
52    0.750000
53    1.000000
54    0.000000
55    0.500000
56    0.666667
57    0.500000
61    1.000000
65    1.000000
67    1.000000
82    1.000000
Name: Survived, dtype: float64

VERIFICACIÓN DE VALORES ÚNICOS Y CONTEO:
NameLength
12     2
13     2
14     3
15    15
16    26
17    42
18    50
19    64
20    39
21    40
22    38
23    39


### Variable  HasCabinNeighbor

In [ ]:
# --- 10. VARIABLE: HasCabinNeighbor (CABINAS CERCANAS CON FAMILIARES ) ---

def has_cabin_neighbor_feature(df):
    df['HasCabinNeighbor'] = df['Cabin'].notnull().astype(int)
    # ANÁLISIS DE LA RELACIÓN CON SUPERVIVENCIA
    print("RELACIÓN DE 'HasCabinNeighbor' CON LA SUPERVIVENCIA:")
    print(df.groupby('HasCabinNeighbor')['Survived'].mean())
    # VERIFICACIÓN DE CALIDAD
    print("\nVERIFICACIÓN DE VALORES ÚNICOS Y CONTEO:")
    print(df['HasCabinNeighbor'].value_counts())

has_cabin_neighbor_feature(df)

RELACIÓN DE 'HasCabinNeighbor' CON LA SUPERVIVENCIA:
HasCabinNeighbor
0    0.299854
1    0.666667
Name: Survived, dtype: float64

VERIFICACIÓN DE VALORES ÚNICOS Y CONTEO:
HasCabinNeighbor
0    687
1    204
Name: count, dtype: int64


### Variable: TicketPrefix

In [ ]:
# --- 11. VARIABLE: TicketPrefix (PREFIJO DEL TCIKET ) ---

def ticket_prefix_feature(df):
    df['TicketPrefix'] = df['Ticket'].str.extract(r'([A-Za-z]+)\d*')
    df['TicketPrefix'].fillna('Unknown', inplace=True)
    # ANÁLISIS DE LA RELACIÓN CON SUPERVIVENCIA
    print("RELACIÓN DE 'TicketPrefix' CON LA SUPERVIVENCIA:")
    print(df.groupby('TicketPrefix')['Survived'].mean().sort_values(ascending=False))
    # VERIFICACIÓN DE CALIDAD
    print("\nVERIFICACIÓN DE VALORES ÚNICOS Y CONTEO:")
    print(df['TicketPrefix'].value_counts())

ticket_prefix_feature(df)



RELACIÓN DE 'TicketPrefix' CON LA SUPERVIVENCIA:
TicketPrefix
SW         1.000000
SO         1.000000
F          0.666667
PP         0.666667
PC         0.650000
SC         0.538462
P          0.500000
WE         0.500000
C          0.454545
STON       0.444444
Unknown    0.384266
LINE       0.250000
S          0.142857
SOTON      0.117647
W          0.090909
CA         0.071429
A          0.068966
Fa         0.000000
SCO        0.000000
Name: Survived, dtype: float64

VERIFICACIÓN DE VALORES ÚNICOS Y CONTEO:
TicketPrefix
Unknown    661
PC          60
C           33
A           29
STON        18
SOTON       17
CA          14
S           14
SC          13
W           11
F            6
LINE         4
PP           3
P            2
WE           2
SO           1
Fa           1
SCO          1
SW           1
Name: count, dtype: int64


/tmp/ipython-input-305781971.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['TicketPrefix'].fillna('Unknown', inplace=True)


# 2.2 Transformaciones de Variables Existentes y 2.3 Análisis de Interacciones



In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

In [ ]:
# La clase TitanicDatasetPreprocessor nos sirve para preprocesar los datos, ubicados en
# el dataframe del titanic llamado df, lo cual no sirve para alimentar modelos de
# machine learning a través de tres/3 funcionalidades que viene siendo:


# 1. Crear nuevas variables a partir de variables existentes (feature engineering)
# 2. Validar que estas variables estén correctas
# 3. Transformar los datos para que sean númericos y escalables para un modelo de Machine Learning




class TitanicDatasetPreprocessor:
    def __init__(self):
        self.pipeline = None # Con pipeline, podemos guardar el objeto de transformación que aplica el escalado
        self.feature_engineered = False # Con feature engineered, tenemos un flag que indica si las nuevas columnas/variables han sido creadas

    # Validación continua
    # Verificamos que la columna exista, checamos cuantos valores nulos hay, nos aseguramos que la columan sea el tipo de dato correcto
    # y checamos cuantos valores unicos hay.
    # Si no cumple ninguna de las condiciones entoncers nos marca error
    def _validate_feature(self, df, col, expected_dtype=None, max_unique=None):
        assert col in df.columns, f" Columna {col} no fue creada."
        assert df[col].isnull().mean() < 0.2, f" Columna {col} tiene demasiados valores nulos."
        if expected_dtype:
            assert df[col].dtype == expected_dtype, f" {col} debería ser {expected_dtype}, pero es {df[col].dtype}."
        if max_unique:
            assert df[col].nunique() <= max_unique, f" {col} tiene demasiados valores únicos ({df[col].nunique()})."
        print(f" Validación pasada: {col}")

    # Tratamiento de outliers con IQR (investigamos se encuuentran valores atípicos y/o extremos)
    def _treat_outliers(self, df, cols):
        for col in cols:
            if col in df.columns:
                Q1 = df[col].quantile(0.25) # Primer Cuartil
                Q3 = df[col].quantile(0.75) # Tercer Cuartil
                IQR = Q3 - Q1 # Intercuartil
                lower = Q1 - 1.5 * IQR # Limite Inferior
                upper = Q3 + 1.5 * IQR # Limmite Superior
                df[col] = np.clip(df[col], lower, upper) # Capturamos los outliers
                print(f" Outliers tratados en {col}")
        return df

    # Visualización de outliers
    def visualize_outliers(self, df, cols):
        for col in cols:
            if col in df.columns:
                fig, axes = plt.subplots(1, 2, figsize=(12, 4))

                # Boxplot para detectar outliers
                sns.boxplot(x=df[col], ax=axes[0])
                axes[0].set_title(f"Boxplot de {col}")

                # Histograma para ver distribución
                sns.histplot(df[col], bins=30, kde=True, ax=axes[1])
                axes[1].set_title(f"Distribución de {col}")

                plt.tight_layout()
                plt.show()

    # Feature Engineering
    def _feature_engineering(self, df):
        feature_funcs = [
            title_feature, family_size_feature, is_alone_feature, age_group_feature,
            fare_per_person_feature, cabin_deck_feature, cabin_known_feature,
            ticket_frequency_feature, name_length_feature, has_cabin_neighbor_feature,
            ticket_prefix_feature
        ]
        for func in feature_funcs:
            df = func(df)

        # Revisamos que la hayan creado ciertas features
        for col in ["Title", "FamilySize", "IsAlone", "AgeGroup", "FarePerPerson"]:
            self._validate_feature(df, col)

        # Agregamos las interraciones interacciones
        df = self._feature_interactions(df)
        return df

    # Interacciones
    def _feature_interactions(self, df):
        df["Sex*Class"] = df["Sex"].astype(str) + "_" + df["Pclass"].astype(str)
        df["Age*Class"] = pd.cut(df["Age"], bins=[0,12,18,40,60,80], labels=False) * df["Pclass"]
        df["Sex*AgeGroup"] = df["Sex"].astype(str) + "_" + df["AgeGroup"].astype(str)
        df["Fare*Embarked"] = pd.qcut(df["Fare"], 4, labels=False) * df["Embarked"].factorize()[0]
        df["Family*Class"] = df["FamilySize"] * df["Pclass"]

        # Validamos que se hayan creado lass interacciones de features
        for col in ["Sex*Class", "Age*Class", "Sex*AgeGroup", "Fare*Embarked", "Family*Class"]:
            self._validate_feature(df, col)

        return df

    # Fit - Aprende de los parámetros de Transformación
    def fit(self, X, y=None):
        X_proc = self._feature_engineering(X.copy())
        self.feature_engineered = True

        # Separamos todass las variables en dos listas: una lista de variables numéricas y otra de variables categóricas
        num_cols = X_proc.select_dtypes(include=np.number).columns.tolist()
        cat_cols = X_proc.select_dtypes(include="object").columns.tolist()

        # Rellenamos nulos con la mediana y luego escalamos los datos
        num_transformer = Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler())
        ])

        # Rellenamos nulos con la moda y aplicamos One-Hot encoding
        cat_transformer = Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(handle_unknown="ignore"))
        ])

        # Nos aseguramoss de aplicar las transformaciones con sus respectivas columnas/variables
        self.pipeline = ColumnTransformer(transformers=[
            ("num", num_transformer, num_cols),
            ("cat", cat_transformer, cat_cols)
        ])

        self.pipeline.fit(X_proc)
        return self

    # Transform - Aplica las transformaciones a un nuevo dataset
    def transform(self, X):
        if not self.feature_engineered:
            raise RuntimeError("Primero usa fit() en datos de entrenamiento.")
        X_proc = self._feature_engineering(X.copy())
        return self.pipeline.transform(X_proc)


    # Combinación de Fit y de Transform
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
